# Pipeline 
Try this online https://colab.research.google.com/drive/1MPzlkCioSXXaL-EqpInR98P1QAM4lRMR 

# This notebook contains knowledge-based evaluations for the extracted IC genes, the example has been done on LIHC results 

In [37]:
import requests
import re
import numpy as np
import csv
import pandas as pd
import joblib
from tqdm import tqdm_notebook
import time 
import scipy.stats as stats
from openpyxl import load_workbook
from gprofiler import GProfiler

# Pubmed evaluation: are the IC genes enriched in cancer type publications against all genes present in the fused network?

In [2]:
#Functions to extract the number of publications associated to a single term (extract_single), 
#two terms with "or" condition (extract_or) and two terms with "and" condition (extract_and)
def extract_single(one):
    try:
        contains = ''
        while contains == '':
            try:
                #%5Btw%5D is used to make sure that the searching term is included in the title, abstract, other abstract, 
                #MeSH terms, MeSH Subheadings, Publication Types or Substance Names.
                contains = requests.get("https://pubmed.ncbi.nlm.nih.gov/?term=%28{}%5Btw%5D%29".format(one))
                break
            except:
                time.sleep(5)
                continue
            
        m = re.search('<meta name="log_resultcount" content="[0-9]+"', str(contains.content))
        return int(m.group(0)[38:-1])
    except:
        m1 = re.search('<meta name="log_source_db" content=".+" />',str(contains.content))
        if m1 is not None:
            return 1
        if m1 is None:
            return 0
        
def extract_or(*args):
    query = "https://pubmed.ncbi.nlm.nih.gov/?term=%28{}%5Btw%5D%29".format(args[0].replace(" ", "+"))
    for a in args[1:]:
        query += "+OR+%28{}%5Btw%5D%29".format(a.replace(" ", "+"))
    try:
        contains = ''
        while contains == '':
            try:
                contains = requests.get(query)
                break
            except:
                time.sleep(5)
                continue
        m = re.search('<meta name="log_resultcount" content="[0-9]+"', str(contains.content))
        return int(m.group(0)[38:-1])
    except:
        m1 = re.search('<meta name="log_source_db" content=".+" />',str(contains.content))
        if m1 is not None:
            return 1
        if m1 is None:
            return 0
        
def extract_and(*args): 
    query = "https://pubmed.ncbi.nlm.nih.gov/?term=%28{}%5Btw%5D%29".format(args[0].replace(" ", "+"))
    for a in args[1:]:
        query += "+AND+%28{}%5Btw%5D%29".format(a.replace(" ", "+"))
    contains = ''
    while contains == '':
        try:
            contains = requests.get(query)
            break
        except:
            time.sleep(5)
            continue
    m = re.search('<meta name="log_resultcount" content="[0-9]+"', str(contains.content))
    if m is not None:
        return int(m.group(0)[38:-1])
    if m is None:
        m1 = re.search('<meta name="log_source_db" content=".+" />',str(contains.content))
        if m1 is not None:
            return 1
        if m1 is None:
            return 0

In [ ]:
#Results are tested on LIHC IC genes
cancer_type='lihc'

ICgenes = pd.read_csv('./Results/'+str(cancer_type)+'_ICgenes.csv', sep=';', header=None)
all_o = set(ICgenes[0])
all_t = set(ICgenes[1])
dict_o={}
dict_t={}

#Extraction of the number of publications for each gene
for i in tqdm_notebook(all_t):
        dict_t[i] = extract_single(i)
#Extraction of the number of publications for the cancer type
for i in tqdm_notebook(all_o):
        dict_o[i] = extract_single(i)
            
n_t_tot=[]
n_o_tot=[]
tot=[]
common=[]
name1=[]
name2=[]

#Extraction of the number of publications of the two terms (gene, cancer type)
#considering OR/AND conditions
for i,j in dict_o.items():
    for key,value in tqdm_notebook(dict_t.items()):
        n_tot = extract_or(i, key)
        n_common = extract_and(i, key)
        name1.append(key)
        name2.append(i)
        n_t_tot.append(value)
        n_o_tot.append(j)
        tot.append(n_tot)
        if n_common == n_tot:
            common.append(0)
        if n_common != n_tot:
            common.append(n_common)

#Save results in csv file:
#first column contains the gene names, second column the cancer type, 
#third column is the number of publications for each gene, fourth column contains the number of publications for the cancer type
#fifth and sixth columns contain the number of publications for condition OR and AND of the two terms gene/cancer type
df = pd.DataFrame(list(zip(name1,name2,n_t_tot,n_o_tot,tot,common)), columns =['genes', 'cancer','pubmed_genes','pubmed_cancer','OR','AND']) 
df.to_csv('./Results/'+str(t)+'_pubmedresults_ICgenes.csv', index=None, header=None)

In [ ]:
#To compute the enrichment we need to count the cancer-type-related publications for all genes in the LIHC fused network
cancer_type='lihc'

all_genes = pd.read_csv('./Results/'+str(cancer_type)+'_allgenes.csv', sep=';', header=None)
all_o = set(all_genes[0])
all_t = set(all_genes[1])
dict_o={}
dict_t={}

#Extraction of the number of publications for each gene
for i in tqdm_notebook(all_t):
        dict_t[i] = extract_single(i)
#Extraction of the number of publications for the cancer type
for i in tqdm_notebook(all_o):
        dict_o[i] = extract_single(i)
            
n_t_tot=[]
n_o_tot=[]
tot=[]
common=[]
name1=[]
name2=[]

#Extraction of the number of publications of the two terms (gene, cancer type)
#considering OR/AND conditions
for i,j in dict_o.items():
    for key,value in tqdm_notebook(dict_t.items()):
        n_tot = extract_or(i, key)
        n_common = extract_and(i, key)
        name1.append(key)
        name2.append(i)
        n_t_tot.append(value)
        n_o_tot.append(j)
        tot.append(n_tot)
        if n_common == n_tot:
            common.append(0)
        if n_common != n_tot:
            common.append(n_common)

#Save results in csv file:
#first column contains the gene names, second column the cancer type, 
#third column is the number of publications for each gene, fourth column contains the number of publications for the cancer type
#fifth and sixth columns contain the number of publications for condition OR and AND of the two terms gene/cancer type
df = pd.DataFrame(list(zip(name1,name2,n_t_tot,n_o_tot,tot,common)), columns =['genes', 'cancer','pubmed_genes','pubmed_cancer','OR','AND']) 
df.to_csv('./Results/'+str(t)+'_pubmedresults_allgenes.csv', index=None, header=None)

In [31]:
def parse_line(l):
    try:
        s = l.strip().split(",")
        return s[0],s[1],int(s[-4]),int(s[-3]), int(s[-2]), int(s[-1])
    except:
        print(l)

In [32]:
#Read the previously saved files
with open('./Results/'+str(cancer_type)+'_pubmedresults_ICgenes.csv') as f:
    all_lines = [parse_line(l) for l in f.readlines()]
with open('./Results/'+str(cancer_type)+'_pubmedresults_allgenes.csv') as f:
    all_all = [parse_line(l) for l in f.readlines()]

In [33]:
print(len(all_all))
print(len(all_lines))

13012
839


In [34]:
#Extract the number of publications for the two terms (gene/cancer type) with AND condition
def pub_and(all_l):
    y = []
    x = list(range(len(all_l)))
    for i in x:
        y.append([int(x[-1]) for x in all_l])
    return np.array(y),x

In [35]:
y_all,_ = pub_and(all_all)
y_IC,_ = pub_and(all_lines)
k=len(np.nonzero(y_IC[0])[0])
M=len(np.nonzero(y_all[0])[0])
N=len(all_all)
s=len(all_lines)

In [36]:
#Fisher's exact test to demostrate that the IC genes are enriched for cancer-type-specific publications
_, pvalue = stats.fisher_exact([[k, s-k],[M-k, N-(M-k)]])  
print(pvalue)

1.045115648264007e-12


# DrugBank evaluation: are the IC genes enriched in druggable targets against all other genes in the network?

In [29]:
#The file DB_pharmacologically_active.csv is directly downloaded from DrugBank
DrugBank=pd.read_csv("./Supplementary data/DB_pharmacologically_active.csv")

In [19]:
#The count_DrugBank function is used to extract the pharmacologically active targets among the input list of genes
def count_DrugBank(data):
    res=[]
    for i in data:
        if any(DrugBank['Gene Name']==i):
            res.append(DrugBank[DrugBank['Gene Name']==i])
    res=pd.concat(res)
    res=res.reset_index(drop=True)
    return res

In [21]:
DB_ICgenes = count_DrugBank(ICgenes[1])
#Print the total number of actionable IC genes
print(len(set(DB_ICgenes['Gene Name'])))
#Print the total number of IC genes
print(len(set(ICgenes[1])))

69
839


In [25]:
DB_allgenes = count_DrugBank(all_genes[1])
#Print the total number of actionable genes from the fused networl
print(len(set(DB_allgenes['Gene Name'])))
#Print the total number of genes from the fused network
print(len(set(all_genes[1])))

481
13011


In [27]:
#Fisher's exact test to demostrate that the IC genes are enriched in druggable targets
k=len(set(DB_ICgenes['Gene Name']))
s=len(set(ICgenes[1]))
M=len(set(DB_allgenes['Gene Name']))
N=len(set(all_genes[1]))
_, pvalue = stats.fisher_exact([[k, s-k],[(M)-k, (N)-((M)-k)]])  
print(pvalue)

1.5976548100792317e-11


# g:Profiler Enrichment analysis of the IC genes

In [38]:
#Using g:Profiler we perform enrichment analysis on the IC genes
gp = GProfiler(return_dataframe=True)
gp_comm=gp.profile(organism='hsapiens', query=list(ICgenes[1]),no_evidences=False)
gp_comm.to_csv('./Enrichment_'+str(cancer_type)+'_ICgenes.csv')

In [39]:
gp_comm

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
0,REAC,REAC:R-HSA-1430728,Metabolism,4.866241e-87,True,Metabolism,2072,553,315,10594,0.569620,0.152027,query_1,[REAC:0000000],"[NDUFAB1, SLC25A13, PON1, FMO3, MGST1, RPS20, ...","[[REAC], [REAC], [REAC], [REAC], [REAC], [REAC..."
1,GO:BP,GO:0055114,oxidation-reduction process,3.009886e-82,True,"""A metabolic process that results in the remov...",1028,745,204,17906,0.273826,0.198444,query_1,[GO:0008152],"[NDUFAB1, SLC25A13, KDM7A, FMO3, MGST1, IYD, U...","[[TAS, NAS, IEA], [IDA, IBA, IEA], [IEA], [IEA..."
2,GO:MF,GO:0016491,oxidoreductase activity,1.542436e-73,True,"""Catalysis of an oxidation-reduction (redox) r...",768,761,169,18126,0.222076,0.220052,query_1,[GO:0003824],"[NDUFAB1, KDM7A, FMO3, MGST1, IYD, UQCRC1, HSD...","[[NAS], [IDA, IEA], [IBA, IEA], [IDA], [IDA, I..."
3,GO:BP,GO:0046395,carboxylic acid catabolic process,4.026219e-65,True,"""The chemical reactions and pathways resulting...",276,745,101,17906,0.135570,0.365942,query_1,"[GO:0016054, GO:0019752]","[PON1, OTC, ACAA1, HSD17B10, ACAT1, ACACB, BCK...","[[IDA], [IDA], [IMP, IBA, TAS], [TAS], [IMP, I..."
4,GO:BP,GO:0016054,organic acid catabolic process,4.026219e-65,True,"""The chemical reactions and pathways resulting...",276,745,101,17906,0.135570,0.365942,query_1,"[GO:0006082, GO:0044248, GO:0044282, GO:1901575]","[PON1, OTC, ACAA1, HSD17B10, ACAT1, ACACB, BCK...","[[IDA], [IDA], [IMP, IBA, TAS], [TAS], [IMP, I..."
5,GO:BP,GO:0044281,small molecule metabolic process,6.617864e-62,True,"""The chemical reactions and pathways involving...",1987,745,253,17906,0.339597,0.127328,query_1,[GO:0008152],"[NDUFAB1, SLC25A13, PON1, IYD, CPS1, OTC, ITIH...","[[NAS, IEA], [IDA, TAS], [IDA, TAS, IEA], [IDA..."
6,GO:CC,GO:0005739,mitochondrion,3.312159e-60,True,"""A semiautonomous, self replicating organelle ...",1631,774,223,18856,0.288114,0.136726,query_1,"[GO:0005737, GO:0043231]","[NDUFAB1, SLC25A13, MGST1, UQCRC1, CPS1, OTC, ...","[[IDA, ISS, IBA, TAS, NAS, IEA], [IDA, ISS, TA..."
7,GO:BP,GO:0044282,small molecule catabolic process,8.601402e-60,True,"""The chemical reactions and pathways resulting...",447,745,119,17906,0.159732,0.266219,query_1,"[GO:0009056, GO:0044281]","[PON1, OTC, ACAA1, HAGH, HSD17B10, ACAT1, ACAC...","[[IDA], [IDA], [IMP, IBA, TAS], [IEA], [TAS], ..."
8,GO:BP,GO:0019752,carboxylic acid metabolic process,3.406416e-58,True,"""The chemical reactions and pathways involving...",1066,745,178,17906,0.238926,0.166979,query_1,[GO:0043436],"[NDUFAB1, PON1, IYD, CPS1, OTC, ITIH4, ITIH1, ...","[[NAS, IEA], [IDA, TAS], [IDA, IBA], [IDA, IBA..."
9,GO:BP,GO:0043436,oxoacid metabolic process,5.231939e-57,True,"""The chemical reactions and pathways involving...",1157,745,184,17906,0.246980,0.159032,query_1,[GO:0006082],"[NDUFAB1, PON1, IYD, CPS1, OTC, ITIH4, ITIH1, ...","[[NAS, IEA], [IDA, TAS], [IDA, IBA], [IDA, IBA..."
